## Problem

The stable marriage problem is defined as follows:

Suppose you have N men and N women, and each person has ranked their prospective opposite-sex partners in order of preference.

For example, if N = 3, the input could be something like this:

```python
guy_preferences = {
    'andrew': ['caroline', 'abigail', 'betty'],
    'bill': ['caroline', 'betty', 'abigail'],
    'chester': ['betty', 'caroline', 'abigail'],
}
```

```python
gal_preferences = {
    'abigail': ['andrew', 'bill', 'chester'],
    'betty': ['bill', 'andrew', 'chester'],
    'caroline': ['bill', 'chester', 'andrew']
}
```

Write an algorithm that pairs the men and women together in such a way that no two people of opposite sex would both rather be with each other than with their current partners.

## Solution
Don't be upset if you found this problem challenging: the Gale-Shapley algorithm to solve this ended up earning one its creators a Nobel Prize!

The algorithm is as follows:
At each step, we consider one currently unmarried man. This man goes through his list of partners, proposing to each one. If the woman is unmarried, she agrees automatically. Otherwise, she will consider whether she prefers this new suitor to her current husband. If she does, she "trade up" to the new offer.

With the input above, we begin with Andrew, who proposes to Caroline. Since Caroline is currently available, she says yes, and they get married. 

We go to Bill who has Caroline as her first choice. Unfortunately for Andrew, Caroline prefers Bill, so she dumps Andrew and takes up with Bill.

Andrew goes to her second choice Abigail, to marry, and she agrees. Finally Chester asks Betty to marry. Sadly for Betty, even though Chester is her least favorite, she has no alternative but to agree.

